In [1]:
import os
from dotenv import load_dotenv
# .env 파일 경로 지정
dotenv_path = os.path.join("./", '.env')
print(dotenv_path)
# .env 파일 로드
load_dotenv(dotenv_path)
crtfc_key = os.getenv("DART_API_KEY")  # API Key 입력
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
from dartapi import DartAPI
import pandas as pd
from tqdm import tqdm 
from glob import glob
import json
import xmltodict

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import zipfile
import xml.etree.ElementTree as ET


./.env


In [2]:
def make_local_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [3]:
today = datetime.now().strftime('%Y%m%d')
# three_months_ago = (datetime.now() - timedelta(days=3*30)).strftime('%Y%m%d')
# six_months_ago = (datetime.now() - timedelta(days=6*30)).strftime('%Y%m%d')
# oneyear_months_ago = (datetime.now() - timedelta(days=12*30)).strftime('%Y%m%d')
# twoyear_months_ago = (datetime.now() - timedelta(days=24*30)).strftime('%Y%m%d')
# threeyear_months_ago = (datetime.now() - timedelta(days=36*30)).strftime('%Y%m%d')


In [4]:
api_class = DartAPI(crtfc_key=crtfc_key)

In [10]:
df = api_class.unique_num()
dic = {i[1]:i[0] for i in zip(df['corp_code'], df['corp_name'])}

'../data/unique_corp' 폴더가 이미 존재합니다.


In [121]:

path = '../data/MainMatterReport_usang_20230803_20230919.csv'
usang_df = pd.read_csv(path, encoding='utf-8-sig', usecols=['회사명', '접수일'])
usang_df['corp_code'] = usang_df['회사명'].map(dic)
usang_dict = {i[1]:i[0] for i in zip(usang_df['corp_code'], usang_df['회사명'])}

In [122]:
## api 가져오기 및 접수일 merge
usang_df_2 = api_class.usang(today, "2023-08-03", usang_dict)
usang_df_2['접수일'] = usang_df['접수일']
usang_df_2.to_csv(path, encoding='utf-8-sig', index=False)

100%|██████████| 63/63 [00:09<00:00,  6.48it/s]


In [124]:
## 보고서 존재 유부 체크
path = "../data/MainMatterReport_usang_20230803_20230919.csv"
usang_df_2 = pd.read_csv(path, encoding='utf-8-sig')

rcept_no = [os.path.basename(i).split('.')[0] for i in glob("../data/doc/*.xml")]
usang_df_2['보고서유무'] = usang_df_2['rcept_no'].apply(lambda x: True if str(x) in rcept_no else False)
usang_df_2.to_csv(path, encoding='utf-8-sig', index=False)

In [8]:
path = "../data/MainMatterReport_usang_20150101_20230919.csv"
usang_df_2 = pd.read_csv(path, encoding='utf-8-sig')
print(len(usang_df_2))

usang_df_2.head()

4675


,rcept_no,corp_cls,corp_code,corp_name,nstk_ostk_cnt,nstk_estk_cnt,fv_ps,bfic_tisstk_ostk,bfic_tisstk_estk,fdpp_fclt,...,fdpp_dtrp,fdpp_ocsa,fdpp_etc,ic_mthn,ssl_at,ssl_bgd,ssl_edd,접수일,보고서유무,parser 에러여부
0,20230914000187,N,1695498,이브이파킹서비스,"1,666,667",-,100,"15,266,666",-,-,...,-,-,-,제3자배정증자,N,-,-,2023-09-15,True,True
1,20230828000410,K,1227039,브릿지바이오,"600,600",-,500,"20,061,690","4,635,865",-,...,-,-,-,제3자배정증자,N,-,-,2023-09-15,True,True
2,20230823000377,N,578130,아퓨어스,"3,679,114",-,500,"38,407,138",-,-,...,-,-,-,제3자배정증자,N,-,-,2023-09-15,True,True
3,20230823000316,N,1185867,퓨쳐메디신,"242,888",-,500,"10,110,317",-,-,...,-,-,-,제3자배정증자,N,-,-,2023-09-15,True,True
4,20230825000523,K,139153,에코플라스틱,"7,000,000",-,500,"34,169,370",-,-,...,-,"27,230,000,000",-,주주배정후 실권주 일반공모,Y,20230828,20231101,2023-08-22,True,True


In [13]:
## parser 에러여부
## 보고서 종류

save_path = "../data/doc/"
usang_df_2["parser 에러여부"] = True
usang_df_2["보고서 종류"] = "-"

for i in tqdm([i for i in zip(usang_df_2['rcept_no'], usang_df_2.index, usang_df_2['ic_mthn'])]):
    
    rcept_no = i[0]
    index = i[1]
    capi_meth = i[2]

    # XML 파일 경로
    xml_file_path = os.path.join(save_path, "%s.xml"%rcept_no)
    # XML 데이터 파싱
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        # if capi_meth in ["주주배정증자", "주주배정후 실권주 일반공모", "제3자배정증자"]:
        report = root.findall(".//TITLE")[0].text
        report = report.replace(" ", "")
        usang_df_2.at[index, "보고서 종류"] = report
   
    except Exception as e:
        usang_df_2.at[index, "parser 에러여부"] = False


usang_df_2.to_csv(path, encoding='utf-8-sig', index=False)

100%|██████████| 4675/4675 [00:01<00:00, 3924.02it/s]


In [146]:
## merge
path = "../data/MainMatterReport_usang_20150101_20230803.csv"
df_1 = pd.read_csv(path, encoding='utf-8-sig')
path = "../data/MainMatterReport_usang_20230803_20230919.csv"
df_2 = pd.read_csv(path, encoding='utf-8-sig')
df = pd.concat([df_1, df_2], axis=0)
df = df.sort_values(['접수일'], ascending=False)
path = "../data/MainMatterReport_usang_20150101_20230919.csv"
df.to_csv(path, encoding='utf-8-sig', index=False)

In [116]:
pd.DataFrame(usang_df_2['ic_mthn'].value_counts()).reset_index()

,index,ic_mthn
0,제3자배정증자,888
1,주주배정후 실권주 일반공모,65
2,주주배정증자,57
3,일반공모증자,48
4,주주우선공모증자,7
